## 0. 환경 준비 (설치/마운트)
- `timm`과 헤드리스 OpenCV는 설치 권장
- Google Drive 마운트 후 `/content/drive` 경로 사용


In [ ]:
# 필요한 패키지 설치 (Colab 기본에는 torch/torchvision/numpy/tqdm/cv2 포함)
!pip install -q timm
!pip install -q opencv-python-headless

from google.colab import drive
drive.mount('/content/drive')  # /content/drive/MyDrive 접근


Mounted at /content/drive


## 1. 라이브러리 임포트

In [ ]:
import os, json, time
import numpy as np
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from torchvision import transforms
from tqdm import tqdm
import timm


## 2. 데이터 압축 해제 유틸 및 실행
### 입력
- `.tar` 내부에 `*.zip.part0`가 포함되어 있다고 **가정**합니다.
- 각 `zip.part0`만 unzip하면 나머지 분할 파트가 함께 풀립니다.

### 출력 폴더
- `/content/training_set`, `/content/validation_set`, `/content/tline_label`, `/content/vline_label`


In [ ]:
TS = "/content/drive/MyDrive/TS_KS.tar"
VS = "/content/drive/MyDrive/VS_KS.tar"
TL = "/content/drive/MyDrive/TL_LINE.tar"
VL = "/content/drive/MyDrive/VL_LINE.tar"

def fast_extract_tar_and_zip_recursive(tar_path, output_folder):
    """
    tar_path: .tar 파일 경로(Drive)
    output_folder: 최종 unzip 결과물이 들어갈 디렉토리
    동작:
      1) /content/temp.tar 로 복사 → temp_dir로 전체 해제
      2) temp_dir에서 *.zip.part0 파일 검색 → 각각을 output_folder로 unzip(-j로 경로 무시)
      3) temp_dir 정리
    주의: .tar 내부 구조가 zip 파트들을 포함한다고 가정합니다.
    """
    temp_dir = "/content/temp_extract"
    os.makedirs(temp_dir, exist_ok=True)

    # .tar를 로컬로 복사 후 해제 (권한/경로 이슈 최소화)
    !cp "{tar_path}" /content/temp.tar
    !tar -xf /content/temp.tar -C "{temp_dir}"
    !rm /content/temp.tar

    # .zip.part0 파일 목록 수집
    zip_part0_paths = []
    for root, _, files in os.walk(temp_dir):
        for f in files:
            if f.endswith(".zip.part0"):
                zip_part0_paths.append(os.path.join(root, f))

    if not zip_part0_paths:
        print(f"⚠️ {tar_path} 안에서 zip.part0를 찾을 수 없습니다.")
        !rm -rf "{temp_dir}"
        return

    # 각 분할 zip의 part0만 unzip하면 나머지 part들이 함께 풀림
    for zp_path in zip_part0_paths:
        !unzip -q -j "{zp_path}" -d "{output_folder}" 2>/dev/null

    # 임시 디렉토리 정리
    !rm -rf "{temp_dir}"

# 출력 폴더 생성(존재해도 무시)
!mkdir -p /content/training_set /content/validation_set /content/tline_label /content/vline_label

# 실제 압축 해제 실행
fast_extract_tar_and_zip_recursive(TS, "/content/training_set")
print('training_set completed')
fast_extract_tar_and_zip_recursive(VS, "/content/validation_set")
print('validation_set completed')
fast_extract_tar_and_zip_recursive(TL, "/content/tline_label")
print('t label completed')
fast_extract_tar_and_zip_recursive(VL, "/content/vline_label")
print('v label completed')
print("✅ 모든 데이터셋 해제 완료!")


training_set completed
validation_set completed
t label completed
v label completed
✅ 모든 데이터셋 해제 완료!


## 3. 전처리 유틸 함수
- **Letterbox + padding mask 생성**
- **폴리라인 두께**(이미지 크기 기반 동적)
- **정규화 길이(feature)** 계산


In [ ]:
def letterbox_with_padmask(imgC, out_h=224, out_w=224):
    """
    비율 유지(letterbox) 리사이즈 + 중앙 정렬 + 남는 영역 패딩(0) + 패딩 마스크(255=패딩, 0=실제)
    입력: imgC (H,W,C)  C=4 (RGB + target_mask)
    출력:
      - img_lb: (out_h, out_w, C) letterbox된 이미지
      - pad_mask: (out_h, out_w) 패딩영역=255, 실제=0 (학습 시 보조 채널로 사용)
    """
    h, w = imgC.shape[:2]
    scale = min(out_w / w, out_h / h)
    nh, nw = int(round(h * scale)), int(round(w * scale))
    resized = cv2.resize(imgC, (nw, nh), interpolation=cv2.INTER_LINEAR)

    img_lb = np.zeros((out_h, out_w, imgC.shape[2]), dtype=resized.dtype)
    pad_mask = np.ones((out_h, out_w), dtype=np.uint8)  # 기본=1(패딩)

    top = (out_h - nh) // 2
    left = (out_w - nw) // 2

    img_lb[top:top + nh, left:left + nw, :] = resized
    pad_mask[top:top + nh, left:left + nw] = 0          # 실제 영역=0

    pad_mask = (pad_mask * 255).astype(np.uint8)
    return img_lb, pad_mask

def dynamic_thickness(h, w, k=0.004):
    """이미지 크기에 따라 폴리라인 두께를 동적으로 산정(최소 1px)"""
    return max(1, int(k * min(h, w)))

def poly_length_norm(all_x, all_y, w, h):
    """
    폴리라인 길이를 (W,H)로 정규화한 총 길이
    - all_x/ all_y: 폴리라인 정점 좌표열
    - (x/W, y/H)로 스케일 후 인접 세그먼트 길이의 합
    """
    pts = np.stack([np.array(all_x, np.float32) / w,
                    np.array(all_y, np.float32) / h], axis=1)
    if len(pts) < 2:
        return 0.0
    seg = np.diff(pts, axis=0)
    return float(np.linalg.norm(seg, axis=1).sum())


## 4. Dataset 정의 (5채널 입력)
- **RGB(3) + target_mask(1) + padding_mask(1) = 5채널**
- VIA JSON의 `region_attributes.chi_height_m`를 정답으로 사용
- 학습셋에서 **길이/타깃 통계**를 추정해 z-정규화, 검증은 그 통계 고정 사용(데이터 누수 방지)


In [ ]:
class ImageMaskHeightDataset(Dataset):
    def __init__(self, image_dir, meta_dir, transform=None,
                 out_size=(224, 224), compute_stats=False, stats=None):
        self.image_dir = image_dir
        self.meta_dir = meta_dir
        self.transform = transform
        self.out_h, self.out_w = out_size
        self.samples = []                     # (img_path, all_x, all_y, height_y)
        self._feat_list, self._y_list = [], []

        # 메타(JSON) 스캔: VIA 형식 가정 (filename, regions[{shape_attributes, region_attributes}])
        for meta_file in os.listdir(meta_dir):
            if not meta_file.endswith('.json'):
                continue
            with open(os.path.join(meta_dir, meta_file), 'r') as f:
                meta = json.load(f)
            for _, v in meta.items():
                img_path = os.path.join(image_dir, v['filename'])
                if not os.path.exists(img_path):
                    continue
                for r in v.get('regions', []):
                    # 키 'chi_height_m'가 있는 region만 사용(정답 레이블)
                    if not r.get('region_attributes') or 'chi_height_m' not in r['region_attributes']:
                        continue
                    ax = r['shape_attributes'].get('all_points_x', [])
                    ay = r['shape_attributes'].get('all_points_y', [])
                    if len(ax) < 2:
                        continue
                    y = float(r['region_attributes']['chi_height_m'])
                    self.samples.append((img_path, ax, ay, y))

        # 학습용 통계치(폴리라인 길이, 타깃)의 평균/표준편차를 계산 → 정규화에 사용
        if compute_stats:
            for img_path, ax, ay, y in self.samples:
                img = cv2.imread(img_path)
                if img is None:
                    continue
                h, w = img.shape[:2]
                ln = poly_length_norm(ax, ay, w, h)
                self._feat_list.append(ln)
                self._y_list.append(y)
            self.feat_mean = float(np.mean(self._feat_list))
            self.feat_std  = float(np.std(self._feat_list) + 1e-8)
            self.y_mean    = float(np.mean(self._y_list))
            self.y_std     = float(np.std(self._y_list) + 1e-8)
        else:
            # 검증/테스트는 학습에서 얻은 stats를 그대로 사용(데이터 누수 방지)
            assert stats is not None
            self.feat_mean, self.feat_std, self.y_mean, self.y_std = stats

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, ax, ay, y = self.samples[idx]
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        h, w = img.shape[:2]

        # 타깃 폴리라인을 단일 채널 mask(0/255)로 렌더링 (원본 RGB는 변형하지 않음)
        mask_tgt = np.zeros((h, w), dtype=np.uint8)
        pts_px = np.stack([np.array(ax, np.int32), np.array(ay, np.int32)], axis=1)
        cv2.polylines(mask_tgt, [pts_px], isClosed=False, color=255, thickness=dynamic_thickness(h, w))

        # 4채널(RGB+mask) → letterbox → pad_mask 생성 → 5채널 합치기
        img4 = np.dstack([img, mask_tgt])
        img4_lb, pad_mask = letterbox_with_padmask(img4, self.out_h, self.out_w)
        img5 = np.dstack([img4_lb, pad_mask])

        # 폴리라인 길이(feature)와 타깃을 z-정규화(학습 통계 사용)
        length_norm = poly_length_norm(ax, ay, w, h)
        feat_z = (length_norm - self.feat_mean) / self.feat_std
        y_z    = (y - self.y_mean) / self.y_std

        # 텐서 변환
        if self.transform is not None:
            img_t = self.transform(img5)
        else:
            img_t = torch.from_numpy(img5.transpose(2, 0, 1)).float() / 255.0

        feat_t = torch.tensor([feat_z], dtype=torch.float32)
        y_t    = torch.tensor(y_z, dtype=torch.float32)
        return img_t, feat_t, y_t


## 5. 모델: ConvNeXt 백본 + FiLM + 게이팅 + 확률 헤드
- 입력: `img5`(B, **5**, H, W), `poly_z`(B, 1)
- 출력: `mu_z`, `log_var` (정규화된 공간)
- **alpha**로 게이팅 강도 조절, `log_var`는 안정성 위해 클램프


In [ ]:
class ProbFiLMGate(nn.Module):
    def __init__(self, model_name='convnext_tiny', pretrained=True,
                 in_chans=5, film_dim=128, gate_dim=128, alpha=0.5,
                 logvar_min=-6.0, logvar_max=4.0, drop_path_rate=0.2):
        super().__init__()
        self.alpha = alpha
        self.logvar_min = logvar_min
        self.logvar_max = logvar_max

        # timm 백본: num_classes=0 → 특징벡터 추출용
        self.backbone = timm.create_model(
            model_name, pretrained=pretrained, in_chans=in_chans, num_classes=0,
            drop_path_rate=drop_path_rate
        )
        d = self.backbone.num_features

        # 길이(feature) → FiLM 파라미터(gamma, beta)
        self.len_to_film = nn.Sequential(
            nn.Linear(1, film_dim), nn.ReLU(),
            nn.Linear(film_dim, 2 * d)
        )

        # 게이트: [f, poly_z] → 스칼라 게이트 g → (1 + alpha * tanh(g))
        self.gate_mlp = nn.Sequential(
            nn.Linear(d + 1, 1)
        )
        # 초기화: 안정화 목적(필요시 강화)
        nn.init.zeros_(self.gate_mlp[-1].bias)
        nn.init.zeros_(self.len_to_film[-1].weight)
        nn.init.zeros_(self.len_to_film[-1].bias)

        # 회귀 헤드: [f_film, len_g] → mu_z, log_var
        self.head = nn.Sequential(
            nn.Linear(d + 1, 128), nn.ReLU(),
            nn.Linear(128, 64), nn.ReLU(),
            nn.Linear(64, 2)
        )

    def forward(self, img5, poly_z):
        f = self.backbone(img5)  # [B,d]

        # FiLM: f * (1+gamma) + beta
        film = self.len_to_film(poly_z)              # [B,2d]
        gamma, beta = torch.chunk(film, 2, dim=1)    # [B,d], [B,d]
        f_film = f * (1 + gamma) + beta

        # 길이 게이팅
        g = self.gate_mlp(torch.cat([f, poly_z], dim=1))  # [B,1]
        gate = 1.0 + self.alpha * torch.tanh(g)
        len_g = poly_z * gate

        # 확률 회귀 헤드
        out = self.head(torch.cat([f_film, len_g], dim=1))  # [B,2]
        mu_z, log_var = out[:, 0], out[:, 1]
        # 분산 안정화: log_var 클램프
        log_var = torch.clamp(log_var, self.logvar_min, self.logvar_max)
        return mu_z, log_var


## 6. 손실/지표 함수
- **Gaussian NLL**(평균 사용)
- **RMSE** (넘파이 구현)


In [ ]:
def gaussian_nll(mu, log_var, target):
    """정규분포 N(mu, sigma^2) 음의 로그우도(샘플별)"""
    var = torch.exp(log_var)
    return 0.5 * (log_var + (target - mu) ** 2 / var)

def rmse_np(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    return float(np.sqrt(np.mean((y_true - y_pred) ** 2)))


## 7. 학습 루틴
- **warmup_epochs** 동안 `log_var` 역전파 차단으로 분산 안정화
- Plateau 스케줄러로 LR 감소
- Best RMSE 시 체크포인트 저장(`state_dict` + `stats`)


In [ ]:
def train_main():
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    EPOCHS = 40
    BATCH = 16
    LR = 1e-4

    MODEL_NAME = 'convnext_base'
    DROP_PATH = 0.1
    TRAIN_IMG_DIR = "/content/training_set/"; TRAIN_META_DIR = "/content/tline_label/"
    VAL_IMG_DIR   = "/content/validation_set/"; VAL_META_DIR = "/content/vline_label/"
    SAVE_PATH = "/content/drive/MyDrive/M2_model.pth"

    # 5채널 정규화(mean/std): RGB는 ImageNet 통계, mask/pad는 0.5/0.5로 스케일링
    normalize = transforms.Normalize(
        mean=[0.485, 0.456, 0.406, 0.5, 0.5],
        std =[0.229, 0.224, 0.225, 0.5, 0.5]
    )
    tfm = transforms.Compose([transforms.ToTensor(), normalize])

    # Dataset 생성: 학습에서 통계 추정 → 검증은 동일 통계 사용
    train_ds = ImageMaskHeightDataset(TRAIN_IMG_DIR, TRAIN_META_DIR, transform=tfm, compute_stats=True)
    stats = (train_ds.feat_mean, train_ds.feat_std, train_ds.y_mean, train_ds.y_std)
    val_ds   = ImageMaskHeightDataset(VAL_IMG_DIR, VAL_META_DIR, transform=tfm, compute_stats=False, stats=stats)

    # DataLoader: 학습은 shuffle=True, 검증은 False
    tr = DataLoader(train_ds, batch_size=BATCH, shuffle=True, num_workers=2, pin_memory=True)
    va = DataLoader(val_ds,   batch_size=BATCH, shuffle=False, num_workers=2, pin_memory=True)

    # 모델/옵티마/스케줄러
    model = ProbFiLMGate(model_name=MODEL_NAME, pretrained=True, in_chans=5, drop_path_rate=DROP_PATH).to(DEVICE)
    opt = optim.AdamW(model.parameters(), lr=LR, weight_decay=1e-4)
    sched = optim.lr_scheduler.ReduceLROnPlateau(opt, mode='min', factor=0.5, patience=3)

    best_rmse = 1e9
    warmup_epochs = 4  # 초반엔 log_var 역전파 차단으로 안정화
    epoch_times = []

    for ep in range(1, EPOCHS + 1):
        ep_start = time.perf_counter()

        # ── Train ───────────────────────────────────────────
        model.train()
        tr_loss = 0.0
        pbar_tr = tqdm(tr, desc=f"Train {ep}/{EPOCHS}", ncols=100)
        for img5, feat_z, y_z in pbar_tr:
            img5, feat_z, y_z = img5.to(DEVICE), feat_z.to(DEVICE), y_z.to(DEVICE)
            opt.zero_grad()
            mu_z, log_var = model(img5, feat_z)
            # warmup 동안 분산 고정: log_var 그래프 분리(detach)
            nll = gaussian_nll(mu_z, log_var.detach(), y_z) if ep <= warmup_epochs else gaussian_nll(mu_z, log_var, y_z)
            loss = nll.mean()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 5.0)
            opt.step()

            tr_loss += loss.item() * img5.size(0)
            pbar_tr.set_postfix(loss=f"{loss.item():.4f}")

        tr_loss /= len(train_ds)

        # ── Validation ──────────────────────────────────────
        model.eval()
        va_loss = 0.0; preds, gts = [], []
        pbar_va = tqdm(va, desc=f"Valid {ep}/{EPOCHS}", ncols=100, leave=False)
        with torch.no_grad():
            for img5, feat_z, y_z in pbar_va:
                img5, feat_z, y_z = img5.to(DEVICE), feat_z.to(DEVICE), y_z.to(DEVICE)
                mu_z, log_var = model(img5, feat_z)
                nll = gaussian_nll(mu_z, log_var, y_z).mean()
                va_loss += nll.item() * img5.size(0)

                # 정규화 역변환(mu_z → 물리 단위, y_z → 물리 단위)
                mu = mu_z.cpu().numpy() * stats[3] + stats[2]
                y  = y_z.cpu().numpy()  * stats[3] + stats[2]
                preds.extend(mu.tolist()); gts.extend(y.tolist())

        va_loss /= len(val_ds)
        val_rmse = rmse_np(gts, preds)
        sched.step(val_rmse)  # plateau 시 LR 감소

        # 진행 시간 로깅(평균 시간으로 잔여 추정)
        ep_time = time.perf_counter() - ep_start
        epoch_times.append(ep_time)
        avg_time = sum(epoch_times) / len(epoch_times)
        remain = avg_time * (EPOCHS - ep)

        print(
            f"Epoch {ep}/{EPOCHS} | Train NLL: {tr_loss:.4f} | Val NLL: {va_loss:.4f} | Val RMSE: {val_rmse:.3f} | "
            f"Epoch time: {ep_time:.1f}s | Avg: {avg_time:.1f}s | Est. remain: {remain/60:.1f} min"
        )

        # 베스트 모델 저장(백본 가중치+통계치)
        if val_rmse < best_rmse:
            best_rmse = val_rmse
            torch.save({'model': model.state_dict(), 'stats': stats}, SAVE_PATH)
            print(f"✅ Saved: {SAVE_PATH}  (best RMSE {best_rmse:.3f})")

    print(f"\nDone. Best RMSE: {best_rmse:.3f}")

if __name__ == "__main__":
    train_main()


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/354M [00:00<?, ?B/s]

Train 1/40: 100%|████████████████████████████████████| 662/662 [07:52<00:00,  1.40it/s, loss=0.2140]


Epoch 1/40 | Train NLL: 0.2284 | Val NLL: 0.1914 | Val RMSE: 11.647 | Epoch time: 489.3s | Avg: 489.3s | Est. remain: 318.1 min
✅ Saved: /content/drive/MyDrive/M2_model.pth  (best RMSE 11.647)


Train 2/40: 100%|████████████████████████████████████| 662/662 [07:42<00:00,  1.43it/s, loss=0.1733]


...

Epoch 38/40 | Train NLL: -2.9273 | Val NLL: -1.7525 | Val RMSE: 4.232 | Epoch time: 479.3s | Avg: 479.7s | Est. remain: 16.0 min


Train 39/40: 100%|██████████████████████████████████| 662/662 [07:42<00:00,  1.43it/s, loss=-2.9582]


Epoch 39/40 | Train NLL: -2.9286 | Val NLL: -1.7627 | Val RMSE: 4.283 | Epoch time: 478.5s | Avg: 479.6s | Est. remain: 8.0 min


Train 40/40: 100%|██████████████████████████████████| 662/662 [07:44<00:00,  1.43it/s, loss=-2.9471]


Epoch 40/40 | Train NLL: -2.9364 | Val NLL: -1.7644 | Val RMSE: 4.098 | Epoch time: 480.1s | Avg: 479.7s | Est. remain: 0.0 min
✅ Saved: /content/drive/MyDrive/M2_model.pth  (best RMSE 4.098)

Done. Best RMSE: 4.098


> epoch = 40에서 RMSE≈4.098이 관측되었다(실행 환경/시드에 따라 변동 가능).

## 8. 평가/추론 (선형 보정 포함)
- 체크포인트(`state_dict` + `stats`) 로드
- `mu`(평균)과 `sigma`(불확실성)를 물리 단위로 역변환
- 선택적으로 `a,b` 선형 보정


In [ ]:
DEFAULT_CALIB = {"a": 1.0036148953447719, "b": -0.5619141258995768, "tau": 2.5370064952492575}

@torch.no_grad()
def eval_from_model_path(ckpt_path,
                         VAL_IMG_DIR="/content/validation_set/",
                         VAL_META_DIR="/content/vline_label/",
                         model_name="convnext_base",
                         in_chans=5,
                         drop_path_rate=0.1,
                         batch_size=16,
                         num_workers=2):
    """
    저장된 체크포인트(가중치+통계)를 불러와 검증셋 RMSE 산출
    - preds: 정규화 역변환된 평균(mu)
    - sigmas: 예측 불확실성(y_std * sqrt(exp(log_var)))
    - (선택) 선형 보정 y' = a*pred + b 적용 시 RMSE(mu-cal) 출력
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    ckpt  = torch.load(ckpt_path, map_location="cpu")
    state = ckpt.get("model", ckpt)
    stats = ckpt["stats"]                       # (feat_mean, feat_std, y_mean, y_std)
    y_mean, y_std = stats[2], stats[3]

    calib = ckpt.get("calib", DEFAULT_CALIB)
    a, b, tau = float(calib["a"]), float(calib["b"]), float(calib["tau"])

    normalize = transforms.Normalize(mean=[0.485,0.456,0.406,0.5,0.5],
                                     std =[0.229,0.224,0.225,0.5,0.5])
    tfm = transforms.Compose([transforms.ToTensor(), normalize])

    val_ds = ImageMaskHeightDataset(VAL_IMG_DIR, VAL_META_DIR, transform=tfm, compute_stats=False, stats=stats)
    va = DataLoader(val_ds, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True)

    # 평가 시에는 pretrained=False (학습된 state로 덮어쓰기 때문)
    model = ProbFiLMGate(model_name=model_name, pretrained=False, in_chans=in_chans, drop_path_rate=drop_path_rate).to(device)
    model.load_state_dict(state, strict=False)
    model.eval()

    preds, sigmas, gts = [], [], []
    for img5, feat_z, y_z in va:
        img5, feat_z, y_z = img5.to(device), feat_z.to(device), y_z.to(device)
        mu_z, log_var = model(img5, feat_z)
        # 정규화 역변환
        mu_m    = (mu_z.cpu().numpy() * y_std) + y_mean
        sigma_m =  y_std * np.sqrt(np.exp(log_var.cpu().numpy()))
        y_m     = (y_z.cpu().numpy()  * y_std) + y_mean
        preds.extend(mu_m.tolist()); sigmas.extend(sigma_m.tolist()); gts.extend(y_m.tolist())

    preds   = np.array(preds)
    sigmas  = np.array(sigmas)
    gts     = np.array(gts)

    rmse = lambda a_, b_: float(np.sqrt(np.mean((a_ - b_)**2)))
    rmse_base   = rmse(preds, gts)
    rmse_mu_cal = rmse(a * preds + b, gts)

    print(f"RMSE(base):  {rmse_base:.3f}")
    print(f"RMSE(mu-cal): {rmse_mu_cal:.3f}")

if __name__ == "__main__":
    eval_from_model_path(
        ckpt_path="/content/drive/MyDrive/M2_model.pth",
        VAL_IMG_DIR="/content/validation_set/",
        VAL_META_DIR="/content/vline_label/",
        model_name="convnext_base",
    )


RMSE(base):  4.098
RMSE(mu-cal): 4.091


### 참고
- 예: `RMSE(base) ≈ 4.098 → 선형 보정 후 RMSE(mu-cal) ≈ 4.091`
- 실행/시드/하드웨어에 따라 수치는 변동될 수 있습니다.
